# Extending SpikeInterface for Calcium Imaging

This notebook demonstrates how to extend the SpikeInterface framework for use with calcium imaging data. We'll show how to:
- Load imaging data from TIFF files
- Explore and visualize the data
- Cache data for efficient processing
- Run Suite2P extraction on the imaging data

SpikeInterface provides a powerful framework for handling extracellular electrophysiology data, and this library extends those concepts to calcium imaging workflows.

In [ ]:
import photon_mosaic_demo.widgets as pmw
from photon_mosaic_demo.extractors import read_scan_image_tiff
import matplotlib.pyplot as plt
from photon_mosaic_demo.extraction.suite2p_extraction import suite2p_extraction
%matplotlib widget


## Load Imaging Data

First, we'll load a TIFF file containing calcium imaging data. The `read_scan_image_tiff` function allows us to specify the channel and plane we want to extract.

In [ ]:
imaging = read_scan_image_tiff(file_path="C:/Users/ariellel/data/spont_00001.tif", channel_name="Channel 1", plane_name='0')

## Inspect the Imaging Object

The imaging object provides a summary of the loaded data, similar to how SpikeInterface recording objects work.

In [ ]:
imaging

## Access Imaging Data

The imaging object supports lazy loading, allowing you to access specific frame ranges without loading the entire dataset into memory. This is particularly useful for large imaging sessions.

In [ ]:
print(f"Number of frames {imaging.get_num_frames()}")
print(f"Image shape {imaging.image_shape}")
video_snippet = imaging.get_series(start_frame=0, end_frame=10) # demonstrating lazy access
print(f"Video snippet shape {video_snippet.shape}")


## Visualize the Average Image

We can compute and display the average image across all frames to get a sense of the field of view and visible structures.

In [ ]:
avg_image = imaging.get_average_image()
plt.figure()
plt.imshow(avg_image, vmax=50)

## Interactive Visualization

The library provides interactive widgets for exploring the imaging time series, making it easy to scrub through frames and identify features of interest.

In [ ]:
w = pmw.plot_imaging_series(imaging, backend="ipywidgets")

## Cache Data for Processing

To enable efficient processing by downstream tools like Suite2P, we can save the imaging data in a binary format. This creates a cached version optimized for fast access.

In [ ]:
image_cache = imaging.save(folder_path="tests", overwrite=True, n_jobs=1)

## Locate the Binary File

The cached data is stored as a binary `.raw` file. We'll locate this file to pass it to Suite2P for cell segmentation and signal extraction.

In [ ]:
import glob
binary_file = glob.glob(image_cache.to_dict()["kwargs"]["folder_path"] + "/*.raw")[0]

## Run Suite2P Extraction

Finally, we run Suite2P's segmentation and extraction pipeline on our cached imaging data. This will identify ROIs (regions of interest) and extract fluorescence traces, which can then be further analyzed using SpikeInterface-style tools.

In [ ]:
Lx, Ly = imaging.image_shape[0], imaging.image_shape[1]
frames=imaging.get_num_frames()
suite2p_extraction(Lx=Lx, Ly=Ly, n_frames=frames,filename=binary_file)